## [분야1] 완전 자율 네트워크 운영을 위한 초단기 네트워크 트래픽 예측 베이스라인 코드
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.

### 추론 실행 환경
1. python 3.10 환경
2. torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
    - 최신 버전입니다.

코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 필요 라이브러리를 직접 설치합니다.
2. 추론용 코드 작성
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. aif.submit() 함수를 호출하여 최종 결과를 제출하는 파트입니다.
    - 마이 페이지에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
    - 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [1]:
!pip install -U aifactory

In [2]:
# 자신의 모델에 필요한 추가 라이브러리 설치
!pip install numpy pandas pytorch-lightning scikit-learn
!pip install torch torchvision torchaudio

In [3]:
!pip install lightning

In [4]:
!pip install joblib

### 2. 추론 환경의 기본 경로 구조
#### 제출 시 주의사항

1. 테스트 데이터 경로: /aif/data/test_inputs.pkl
   - 채점에 사용될 테스트 입력 데이터는 이 디렉토리 안에 test_inputs.pkl이라는 이름으로 고정되어 제공됩니다.
2. 모델 및 자원 경로: 예시 : ./model/
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)
3. 제출 파일은 submission.pkl로 저장
4. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
5. return 할 결과물과 양식에 유의합니다.

In [5]:
import os
import numpy as np
import pandas as pd
import glob
import joblib
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import pickle
import json
import torch.nn.functional as F

# 0. 재현성을 위한 시드 고정
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# === 설정 ===
SEQ_LEN = 100
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
DROPOUT = 0.2
D_MODEL = 64
D_FF = 128
NUM_KERNELS = 6
TOP_K = 5
E_LAYERS = 2
MODEL_NAME = "traffic_timesnet_model"

# 디바이스 설정 (GPU 우선 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. 테스트 데이터셋 클래스

class TrafficTestDataset(Dataset):
    def __init__(self, x_data, seq_len=100):
        self.data = x_data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.data[idx])

# 2. LSTM 모델 정의

# === TCN 구조 ===
def FFT_for_Period(x, k=2):
    """FFT를 사용하여 주요 주기를 찾는 함수"""
    # x shape: [B, T, N]
    xf = torch.fft.rfft(x, dim=1)
    frequency_list = abs(xf).mean(0).mean(-1)
    frequency_list[0] = 0

    # 유효한 주파수만 선택 (0이 아닌 값들)
    valid_freqs = frequency_list[1:]  # DC component 제외
    if len(valid_freqs) == 0:
        # 유효한 주파수가 없는 경우 기본값 사용
        period = torch.tensor([x.shape[1] // 2, x.shape[1] // 4])[:k]
        period_weight = torch.ones(k)
        return period.numpy(), period_weight

    # top-k 선택시 유효한 범위 내에서만 선택
    k = min(k, len(valid_freqs))
    _, top_indices = torch.topk(valid_freqs, k)
    top_list = top_indices + 1  # DC component를 제외했으므로 +1

    # period 계산 (0 division 방지)
    period = []
    for idx in top_list:
        p = x.shape[1] // max(idx.item(), 1)
        period.append(max(p, 1))  # period는 최소 1

    period = np.array(period)
    period_weight = frequency_list[top_list]  # [k] 차원

    return period, period_weight

class TimesBlock(nn.Module):
    def __init__(self, seq_len, pred_len, top_k, d_model, d_ff, num_kernels):
        super(TimesBlock, self).__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.k = top_k

        # parameter-efficient design
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=d_model, out_channels=d_ff,
                      kernel_size=1, bias=False),
            nn.ReLU(),
            nn.Conv1d(in_channels=d_ff, out_channels=d_model,
                      kernel_size=1, bias=False)
        )

        # multi-scale convolution
        self.inception = nn.ModuleList([
            nn.Conv2d(1, 1, kernel_size=(1, k), padding=(0, k//2))
            for k in [3, 5, 7, 9]
        ])

    def forward(self, x):
        B, T, N = x.shape
        period_list, period_weight = FFT_for_Period(x, self.k)

        res = []
        for i in range(self.k):
            period = period_list[i]

            # period가 너무 작거나 큰 경우 처리
            if period <= 0:
                period = 1
            if period > T:
                period = T

            # padding을 현재 시퀀스 길이 기준으로 계산
            if T % period != 0:
                length = ((T // period) + 1) * period
                padding_size = length - T
                padding = torch.zeros([B, padding_size, N]).to(x.device)
                out = torch.cat([x, padding], dim=1)
            else:
                length = T
                out = x

            # 안전한 reshape를 위한 크기 확인
            if length % period != 0:
                # period로 나누어떨어지도록 조정
                target_length = (length // period) * period
                out = out[:, :target_length, :]
                length = target_length

            # reshape
            H = length // period
            if H > 0 and period > 0:
                out = out.reshape(B, H, period, N).permute(0, 3, 1, 2).contiguous()
                # 2D conv: from 1d Variation to 2d Variation
                out = out.reshape(B * N, 1, H, period)

                # multi-scale convolution
                conv_out = []
                for conv in self.inception:
                    conv_out.append(conv(out))
                out = torch.stack(conv_out, dim=-1).mean(-1)

                # reshape back
                out = out.reshape(B, N, H, period).permute(0, 2, 3, 1).contiguous()
                out = out.reshape(B, H * period, N)

                # 원래 길이로 자르기
                out = out[:, :T, :]
            else:
                # period가 유효하지 않은 경우 원본 반환
                out = x

            res.append(out)

        if len(res) > 0:
            res = torch.stack(res, dim=-1)

            # adaptive aggregation
            if period_weight.dim() > 1:
                period_weight = period_weight.mean(0)  # [k] 차원으로 축소
            period_weight = F.softmax(period_weight, dim=0)
            period_weight = period_weight.view(1, 1, 1, -1).repeat(B, T, N, 1)
            res = torch.sum(res * period_weight, -1)
        else:
            res = x

        # residual connection
        res = res + x
        return res

class Model(nn.Module):
    def __init__(self, seq_len, pred_len, enc_in, d_model, d_ff, num_kernels, top_k, e_layers, dropout):
        super(Model, self).__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.model = nn.ModuleList([TimesBlock(seq_len, pred_len, top_k, d_model, d_ff, num_kernels)
                                    for _ in range(e_layers)])
        self.enc_embedding = nn.Linear(enc_in, d_model)
        self.layer_norm = nn.LayerNorm(d_model)
        self.predict_linear = nn.Linear(self.seq_len, self.pred_len)
        self.projection = nn.Linear(d_model, 1, bias=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: [Batch, Input length, Channel]
        enc_out = self.enc_embedding(x)
        enc_out = self.layer_norm(enc_out)

        for i in range(len(self.model)):
            enc_out = self.layer_norm(self.model[i](enc_out))

        enc_out = self.dropout(enc_out)

        # predictor
        enc_out = enc_out.permute(0, 2, 1)  # [B, C, T]
        enc_out = self.predict_linear(enc_out)  # [B, C, pred_len]
        enc_out = enc_out.permute(0, 2, 1)  # [B, pred_len, C]

        # project to output
        dec_out = self.projection(enc_out)  # [B, pred_len, 1]

        return dec_out[:, -1, :]  # return last prediction

# === Lightning Module ===
class TimesNetRegressor(pl.LightningModule):
    def __init__(self, input_size, seq_len, d_model, d_ff, num_kernels, top_k, e_layers, lr, dropout):
        super().__init__()
        self.model = Model(
            seq_len=seq_len,
            pred_len=1,  # 1-step prediction
            enc_in=input_size,
            d_model=d_model,
            d_ff=d_ff,
            num_kernels=num_kernels,
            top_k=top_k,
            e_layers=e_layers,
            dropout=dropout
        )
        self.criterion = nn.MSELoss()
        self.lr = lr

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


# 3. 저장된 모델과 스케일러 로드
def load_model_components(model_name=MODEL_NAME,
                         model_path="./model"):
    # 메타데이터 로드
    with open(f'{model_path}/{model_name}_meta.json', 'r') as f:
        metadata = json.load(f)

    # X 스케일러 로드
    with open(f'{model_path}/{model_name}_scaler.pkl', 'rb') as f:
        x_scaler = pickle.load(f)

    # Y 스케일러 로드 (있다면)
    y_scaler_path = f'{model_path}/{model_name}_y_scaler.pkl'

    with open(y_scaler_path, 'rb') as f:
        y_scaler = pickle.load(f)


    # 모델 로드
    input_size = metadata.get('input_size', 27)
    hidden_channels = metadata.get('hidden_channels', [64, 64, 64])
    # num_layers = metadata.get('num_layers', 1)
    dropout = metadata.get('dropout', 0.2)
    lr = metadata.get('lr', 0.001)
    seq_len = metadata.get('seq_len', 100)

    model = TimesNetRegressor(
        input_size=27,
        seq_len=SEQ_LEN,
        d_model=D_MODEL,
        d_ff=D_FF,
        num_kernels=NUM_KERNELS,
        top_k=TOP_K,
        e_layers=E_LAYERS,
        lr=LEARNING_RATE,
        dropout=DROPOUT
    )
    model.load_state_dict(torch.load(f'{model_path}/{model_name}.pth', map_location=device))
    model.to(device)
    model.eval()

    return model, x_scaler, y_scaler, metadata

# 4. 테스트 데이터 로드 및 전처리
test_data = joblib.load("/aif/data/test_inputs.pkl")

# 모델 컴포넌트 로드
model_name = MODEL_NAME
model, x_scaler, y_scaler, metadata = load_model_components(model_name)

# 5. 테스트 데이터 전처리
processed_test_data = []

# test_data가 (163854, 100, 27) 형태의 numpy array인 경우
if isinstance(test_data, np.ndarray) and len(test_data.shape) == 3:
    # 효율적인 방법: 전체를 한번에 처리
    n_samples, original_seq_len, n_features = test_data.shape

    # 2D로 reshape하여 스케일링
    test_data_2d = test_data.reshape(-1, n_features)
    test_data_scaled_2d = x_scaler.transform(test_data_2d)
    test_data_scaled = test_data_scaled_2d.reshape(n_samples, original_seq_len, n_features)

    # 시퀀스 길이 조정 (SEQ_LEN에 맞춤)
    seq_len = metadata.get('seq_len', 100)

    for i in range(n_samples):
        sample = test_data_scaled[i]  # (100, 27)

        if seq_len <= original_seq_len:
            # 마지막 seq_len개만 사용
            sequence = sample[-seq_len:]
        else:
            # 패딩이 필요한 경우 (거의 없을 것)
            pad_len = seq_len - original_seq_len
            padding = np.zeros((pad_len, n_features))
            sequence = np.vstack([padding, sample])

        processed_test_data.append(sequence)

else:
    # test_data가 리스트 형태인 경우 (예전 방식)
    for sample in test_data:
        # 각 샘플을 개별적으로 처리
        sample_array = np.array(sample) if not isinstance(sample, np.ndarray) else sample

        # 스케일링 적용
        sample_scaled = x_scaler.transform(sample_array)

        # 시퀀스 길이 확인 및 조정
        seq_len = metadata.get('seq_len', 100)
        if len(sample_scaled) >= seq_len:
            sequence = sample_scaled[-seq_len:]
        else:
            pad_len = seq_len - len(sample_scaled)
            padding = np.zeros((pad_len, sample_scaled.shape[1]))
            sequence = np.vstack([padding, sample_scaled])

        processed_test_data.append(sequence)

# numpy array로 변환
processed_test_data = np.array(processed_test_data)

# 6. 데이터셋 및 데이터로더 생성
test_dataset = TrafficTestDataset(processed_test_data)
BATCH_SIZE = 256
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# 7. 추론 실행
y_pred = []

with torch.no_grad():
    test_pbar = tqdm(test_loader, desc="[base_line]")
    for x_batch in test_pbar:
        x_batch = x_batch.to(device)
        outputs = model(x_batch)  # (batch_size, 1)
        
        # peak_volume 예측값
        outputs_cpu = outputs.cpu().numpy().flatten()  # (batch_size,)
        
        # y_scaler가 있으면 역변환
        if y_scaler is not None:
            outputs_original = y_scaler.inverse_transform(outputs_cpu.reshape(-1, 1))
        else:
            outputs_original = outputs_cpu
        
        y_pred.append(outputs_original)

# 8. 결과 합치기
y_pred = np.concatenate(y_pred, axis=0)

# 9. 결과 저장 (numpy array 형태)
joblib.dump(y_pred, "submission.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '/aif/data/test_inputs.pkl'

### 3. 제출하기 
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 대회 페이지 [베이스라인 코드](https://aifactory.space/task/9162/baseline) 탭에 기재된 가이드라인을 따라 task별로 확인하실 수 있습니다.
- key가 틀리면 제출이 진행되지 않거나 잘못 제출되므로 태스크에 맞는 자기 key를 사용해야 합니다.

In [1]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
aif.submit(model_name="timesnet_test",
           key="f432cb70-58cc-4748-841b-a1b7ba92d5fd"
           )
#-----------------------------------------------------#
print(time.time() - t)

file : task
jupyter notebook
제출 완료
22.970052003860474
